In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Configure IAM Policy in Vertex AI Feature Store


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/feature_store/vertex_ai_feature_store_iam_policy.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fblob%2Fmain%2Fnotebooks%2Fofficial%2Ffeature_store%2Fvertex_ai_feature_store_iam_policy.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/tree/main/notebooks/official/feature_store/vertex_ai_feature_store_iam_policy.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/main/notebooks/official/feature_store/vertex_ai_feature_store_iam_policy.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview

This tutorial demonstrates how to configure an IAM Policy to control access to resources and data stored within Vertex AI Feature Store.

Learn more about [Vertex AI Feature Store](https://cloud.google.com/vertex-ai/docs/featurestore/overview).

The steps performed include:
- Create a BigQuery dataset and set up a `FeatureView` to run periodic sync jobs.
- Provision an online store instance and set up online serving.
- Configure an IAM Policy to manage access controls.

## Getting Started

### Install Vertex AI SDK and other required packages


In [ ]:
! pip3 install --upgrade --user --quiet google-cloud-aiplatform google-cloud-bigquery

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

## Configure IAM Policy in Vertex AI Feature Store

In [ ]:
LOCATION_PARENT = "projects/" + PROJECT_ID + "/locations/" + LOCATION

Configure the necessary service clients:
- [FeaturestoreServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.featurestore_service.FeaturestoreServiceClient)
- [FeatureOnlineStoreAdminServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.feature_online_store_admin_service.FeatureOnlineStoreAdminServiceClient)
- [FeatureRegistryServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.feature_registry_service.FeatureRegistryServiceClient)

In [ ]:
from google.cloud.aiplatform_v1 import (FeatureOnlineStoreAdminServiceClient,
                                        FeatureOnlineStoreServiceClient,
                                        FeatureRegistryServiceClient,
                                        FeaturestoreServiceClient)
from google.cloud.aiplatform_v1.types import \
    feature_online_store_service as feature_online_store_service_pb2
from google.iam.v1 import iam_policy_pb2, policy_pb2

ENDPOINT = LOCATION + "-aiplatform.googleapis.com"
featurestore_service_client = FeaturestoreServiceClient(
    client_options={"api_endpoint": ENDPOINT}
)
feature_online_store_admin_service_client = FeatureOnlineStoreAdminServiceClient(
    client_options={"api_endpoint": ENDPOINT}
)
feature_registry_service_client = FeatureRegistryServiceClient(
    client_options={"api_endpoint": ENDPOINT}
)

Generate sample data for this colab.

In [ ]:
DATASET_ID = "test_data"+"_"+LOCATION.replace('-', '_')  # @param {type:"string"}
TABLE_ID = "tableA"  # @param {type:"string"}

!bq mk --dataset_id={DATASET_ID} --location={LOCATION}
!bq query --nouse_legacy_sql \
"CREATE TABLE {DATASET_ID}.{TABLE_ID} AS (" \
"SELECT * FROM UNNEST(ARRAY<STRUCT<entity_id STRING, feature_timestamp TIMESTAMP, feature1 INT64, feature2 INT64>>[" \
"('test', TIMESTAMP('2024-02-26 08:00:00 UTC'), 10, 20)," \
"('test', TIMESTAMP('2024-02-27 08:00:00 UTC'), 30, 40)," \
"('test', TIMESTAMP('2024-02-28 08:00:00 UTC'), 50, 60)]))"

### Create `FeatureOnlineStore` and `FeatureView` resources

Provision a `FeatureOnlineStore` instance.

In [ ]:
FEATURE_ONLINE_STORE_ID = "test_feature_online_store"

feature_online_store_admin_service_client.create_feature_online_store(
    {
        "parent": LOCATION_PARENT,
        "feature_online_store_id": FEATURE_ONLINE_STORE_ID,
        "feature_online_store": {
            "bigtable": {
                "auto_scaling": {
                    "min_node_count": 1,
                    "max_node_count": 3,
                    "cpu_utilization_target": 50,
                }
            }
        },
    }
).result()

Configure periodic data synchronization from the offline store in BigQuery to the `FeatureOnlineStore` instance.

In [ ]:
FEATURE_VIEW_ID = "test_feature_view"
FEATURE_ONLINE_STORE_RESOURCE_ID = (
    LOCATION_PARENT + "/featureOnlineStores/" + FEATURE_ONLINE_STORE_ID
)
FEATURE_VIEW_RESOURCE_ID = (
    FEATURE_ONLINE_STORE_RESOURCE_ID + "/featureViews/" + FEATURE_VIEW_ID
)
create_feature_view_result = (
    feature_online_store_admin_service_client.create_feature_view(
        parent=FEATURE_ONLINE_STORE_RESOURCE_ID,
        feature_view_id=FEATURE_VIEW_ID,
        feature_view={
            "big_query_source": {
                "uri": f"bq://{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}",
                "entity_id_columns": ["entity_id"],
            },
            "sync_config": {"cron": "0 12 * * *"},
        },
    ).result()
)

## Configure an IAM Policy

Configure IAM Policy for a feature online store.

In [ ]:
# A specific user email or a group email can be used to add members to the IAM Policy.
EXAMPLE_RESOURCE_VIEWER_EMAIL = (
    "google-cloud-eng-fte@google.com"  # @param {type:"string"}
)
EXAMPLE_DATA_VIEWER_EMAIL = "python-sample-owners@google.com"  # @param {type:"string"}
# A Service Account can also be added to the IAM Policy.
EXAMPLE_ADMIN_SERVICE_ACCOUNT = "samples@python-docs-samples-tests.iam.gserviceaccount.com"  # @param {type:"string"}
feature_online_store_admin_service_client.set_iam_policy(
    request=iam_policy_pb2.SetIamPolicyRequest(
        resource=FEATURE_ONLINE_STORE_RESOURCE_ID,
        policy=policy_pb2.Policy(
            bindings=[
                policy_pb2.Binding(
                    role="roles/aiplatform.featurestoreResourceViewer",
                    members=[f"group:{EXAMPLE_RESOURCE_VIEWER_EMAIL}"],
                ),
                policy_pb2.Binding(
                    role="roles/aiplatform.featurestoreDataViewer",
                    members=[f"group:{EXAMPLE_DATA_VIEWER_EMAIL}"],
                ),
                policy_pb2.Binding(
                    role="roles/aiplatform.admin",
                    members=[f"serviceAccount:{EXAMPLE_ADMIN_SERVICE_ACCOUNT}"],
                ),
            ]
        ),
    )
)

Confirm the updated IAM Policy of the feature online store.

In [ ]:
feature_online_store_admin_service_client.get_iam_policy(
    request=iam_policy_pb2.GetIamPolicyRequest(
        resource=FEATURE_ONLINE_STORE_RESOURCE_ID
    )
)

Configure IAM Policy for a feature view.

In [ ]:
# A specific user email or a group email can be used to add members to the IAM Policy.
EXAMPLE_RESOURCE_VIEWER_EMAIL = (
    "google-cloud-eng-fte@google.com"  # @param {type:"string"}
)
EXAMPLE_DATA_VIEWER_EMAIL = "python-sample-owners@google.com"  # @param {type:"string"}
# A Service Account can also be added to the IAM Policy.
EXAMPLE_ADMIN_SERVICE_ACCOUNT = "samples@python-docs-samples-tests.iam.gserviceaccount.com"  # @param {type:"string"}
feature_online_store_admin_service_client.set_iam_policy(
    request=iam_policy_pb2.SetIamPolicyRequest(
        resource=FEATURE_VIEW_RESOURCE_ID,
        policy=policy_pb2.Policy(
            bindings=[
                policy_pb2.Binding(
                    role="roles/aiplatform.featurestoreResourceViewer",
                    members=[f"group:{EXAMPLE_RESOURCE_VIEWER_EMAIL}"],
                ),
                policy_pb2.Binding(
                    role="roles/aiplatform.featurestoreDataViewer",
                    members=[f"group:{EXAMPLE_DATA_VIEWER_EMAIL}"],
                ),
                policy_pb2.Binding(
                    role="roles/aiplatform.admin",
                    members=[f"serviceAccount:{EXAMPLE_ADMIN_SERVICE_ACCOUNT}"],
                ),
            ]
        ),
    )
)

Wait some time for the newly-added IAM policy binding to take effect.

* Note: The policy change typically takes 2 minutes, so this delay may be reduced. See IAM docs for more details.

In [ ]:
# Wait 10 minutes for the newly updated IAM policy binding to become effective.
from time import sleep

sleep(600)

Confirm the updated IAM Policy.

In [ ]:
feature_online_store_admin_service_client.get_iam_policy(
    request=iam_policy_pb2.GetIamPolicyRequest(resource=FEATURE_VIEW_RESOURCE_ID)
)

Run on-demand batch sync.

In [ ]:
sync_response = feature_online_store_admin_service_client.sync_feature_view(
    feature_view=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}/featureViews/{FEATURE_VIEW_ID}"
)

# Wait for sync completion
while True:
    feature_view_sync = feature_online_store_admin_service_client.get_feature_view_sync(
        name=sync_response.feature_view_sync
    )
    if feature_view_sync.run_time.end_time.seconds > 0:
        status = "Succeed" if feature_view_sync.final_status.code == 0 else "Failed"
        print(f"Sync {status} for {feature_view_sync.name}.")
        break
    else:
        print("Sync ongoing, waiting for 30 seconds.")
    sleep(30)

Confirm the status of batch sync.

In [ ]:
feature_online_store_admin_service_client.get_feature_view_sync(
    name=sync_response.feature_view_sync
)

### Start online serving

After the data sync is complete, use the `FetchFeatureValues` API to retrieve the data.

In [ ]:
data_client = FeatureOnlineStoreServiceClient(client_options={"api_endpoint": ENDPOINT})

Read the synced data from feature online store.

In [ ]:
data_client.fetch_feature_values(
    request=feature_online_store_service_pb2.FetchFeatureValuesRequest(
        feature_view=FEATURE_VIEW_RESOURCE_ID,
        data_key=feature_online_store_service_pb2.FeatureViewDataKey(key="test"),
    )
)

## Cleaning up

Cleaning up
To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the cluster you created in this tutorial.

In [ ]:
# Delete feature view
feature_online_store_admin_service_client.delete_feature_view(
    name=FEATURE_VIEW_RESOURCE_ID
)

# Delete online store
feature_online_store_admin_service_client.delete_feature_online_store(
    name=FEATURE_ONLINE_STORE_RESOURCE_ID
)

# Delete test data
!bq rm -f {DATASET_ID}.{TABLE_ID}